# CVE Lakehouse - Bronze Layer: Raw Data Ingestion

**Assignment 1 - DIC 587 Data Intensive Computing**

##  Objective
Ingest and preserve raw CVE vulnerability data from GitHub in its native JSON format using Delta Lake for ACID guarantees.

## Data Source
**CVE v5 Repository**: https://github.com/CVEProject/cvelistV5
- **Description**: Official Common Vulnerabilities and Exposures database
- **Format**: Nested JSON files (CVE v5 schema)
- **Scale**: 200,000+ total vulnerabilities, ~40,000 from 2024
- **Update Frequency**: Live data, updated every ~7 minutes

##  Bronze Layer Implementation

### 1. Data Download & Optimization
- Downloads CVE repository from GitHub (~200MB ZIP)
- **Smart Filtering**: Only copies 2024 CVE files (38,727 vs 200,000+ files)
- **Performance**: 70% reduction in processing time (15 min vs 45+ min)
- Copies files to Unity Catalog volume for Spark access

### 2. Data Quality Gates
- **Count Validation**: Ensures >= 30,000 2024 records
- **Null Checks**: Validates no null CVE IDs
- **Uniqueness**: Confirms all CVE IDs are unique
- **Date Filtering**: Filters by `cveMetadata.datePublished` = 2024

### 3. Delta Lake Features
- **Column Sanitization**: Handles special characters in nested field names
- **Array Serialization**: Converts nested arrays to JSON strings
- **Schema Preservation**: Maintains original JSON structure
- **ACID Transactions**: Guarantees data consistency

## Key Learning Outcomes
- **Bronze Layer Concepts**: Raw data preservation patterns
- **JSON Schema-on-Read**: PySpark's schema inference capabilities
- **Delta Lake**: ACID transactions for data lakes
- **Unity Catalog**: Enterprise data governance and volumes
- **Large-Scale Downloads**: Programmatic data acquisition

## Expected Output
- **Records**: ~40,000 CVE records from 2024
- **Storage**: Unity Catalog volume (`/Volumes/workspace/default/assignment1/bronze`)
- **Format**: Delta Lake with sanitized column names
- **Files**: ~38,727 JSON files processed into 8 Delta partitions

In [0]:
# SOLUTION BLOCK (for reference - implement the TODOs above first!)
# Uncomment and run this cell if you need help completing the Bronze layer

"""
from pyspark.sql import functions as F

# Many CVE v5 exports have metadata under \"cveMetadata.datePublished\"
# Fall back defensively if a different field name appears
date_col = F.col(\"cveMetadata.datePublished\").cast(\"string\")
df_raw = df_raw.withColumn(\"_datePublished_ts\", F.to_timestamp(date_col))

# 2) Filter to 2024 publications
df_2024 = df_raw.filter(F.year(F.col(\"_datePublished_ts\")) == 2024)

# 3) Data quality gates
cnt_all = df_raw.count()
cnt_2024 = df_2024.count()
null_ids = df_2024.filter(F.col(\"cveMetadata.cveId\").isNull()).count()
distinct_ids = df_2024.select(\"cveMetadata.cveId\").distinct().count()

print(f\"Raw total: {cnt_all:,}\")
print(f\"2024 total: {cnt_2024:,}\")
print(f\"Null cveId (2024): {null_ids}\")
print(f\"Distinct cveId (2024): {distinct_ids:,}\")

assert cnt_2024 >= 30000, \"Too few 2024 rows—did the filter or read fail?\"
assert null_ids == 0, \"cveId should not be null\"
assert distinct_ids == cnt_2024, \"cveId expected to be unique in 2024\"

# 4) Write Delta + register table
(df_2024
 .repartition(8)
 .write
 .format(\"delta\")
 .mode(\"overwrite\")
 .save(bronze_path))

spark.sql(f\"DROP TABLE IF EXISTS {table_name}\")
spark.sql(f\"CREATE TABLE {table_name} USING DELTA LOCATION '{bronze_path}'\")

print(\"✅ Bronze Delta written and table registered:\", table_name)

# 5) Verification for screenshots
spark.read.format(\"delta\").load(bronze_path).printSchema()
spark.sql(f\"DESCRIBE DETAIL {table_name}\").show(truncate=False)
"""

'\nfrom pyspark.sql import functions as F\n\n# Many CVE v5 exports have metadata under "cveMetadata.datePublished"\n# Fall back defensively if a different field name appears\ndate_col = F.col("cveMetadata.datePublished").cast("string")\ndf_raw = df_raw.withColumn("_datePublished_ts", F.to_timestamp(date_col))\n\n# 2) Filter to 2024 publications\ndf_2024 = df_raw.filter(F.year(F.col("_datePublished_ts")) == 2024)\n\n# 3) Data quality gates\ncnt_all = df_raw.count()\ncnt_2024 = df_2024.count()\nnull_ids = df_2024.filter(F.col("cveMetadata.cveId").isNull()).count()\ndistinct_ids = df_2024.select("cveMetadata.cveId").distinct().count()\n\nprint(f"Raw total: {cnt_all:,}")\nprint(f"2024 total: {cnt_2024:,}")\nprint(f"Null cveId (2024): {null_ids}")\nprint(f"Distinct cveId (2024): {distinct_ids:,}")\n\nassert cnt_2024 >= 30000, "Too few 2024 rows—did the filter or read fail?"\nassert null_ids == 0, "cveId should not be null"\nassert distinct_ids == cnt_2024, "cveId expected to be unique in 20

In [0]:
# ==============================================================================
# 🥉 BRONZE LAYER - RAW DATA INGESTION WITH SCHEMA PRESERVATION
# ==============================================================================
# Purpose: Ingest CVE v5 JSON data and preserve in Delta Lake format
# Pattern: Download → Filter → Sanitize → Write to Delta
# Output: Raw Bronze Delta table with 2024 CVE records
# ==============================================================================

from pyspark.sql import functions as F, types as T
import re, time

# ==============================================================================
# CONFIGURATION
# ==============================================================================
volume_root      = "/Volumes/workspace/default/assignment1"
volume_cves_dir  = f"{volume_root}/cve/json_files"    # JSON file storage
bronze_path      = f"{volume_root}/bronze"            # Delta output

# Optimize for Databricks Community Edition (single-node cluster)
spark.conf.set("spark.sql.shuffle.partitions", "8")

print("="*70)
print("🥉 BRONZE LAYER - CVE 2024 DATA INGESTION")
print("="*70)
print()
print("🎯 Objective: Ingest 40,000+ CVE records from 2024")
print("💾 Source: CVEProject/cvelistV5 GitHub repository")
print("🔧 Techniques: JSON parsing, date filtering, Delta Lake")
print()

# ==============================================================================
# HELPER FUNCTION: COLUMN NAME SANITIZATION
# ==============================================================================
# Challenge: JSON fields contain special characters (spaces, parentheses, etc.)
# Example: "containers.adp.metrics.Technical Impact" has spaces
# Solution: Replace invalid characters with underscores
# Business Value: Ensures Delta Lake compatibility
# ==============================================================================

INVALID_CHARS = re.compile(r"[ ,;{}()\n\t=]")

def sanitize_path(path: str) -> str:
    """Replace special characters in column names with underscores."""
    return INVALID_CHARS.sub("_", path)

def build_sanitized_columns(df):
    """
    Recursively walk schema and create sanitized column references.
    
    Handles:
    - Nested structs: Flatten with dot notation
    - Arrays of structs: Serialize to JSON strings (preserves structure)
    - Primitive types: Direct column reference
    
    Returns: List of Column expressions with safe aliases
    """
    cols = []
    
    def walk(schema, prefix=""):
        for field in schema.fields:
            name = field.name
            path = f"{prefix}.{name}" if prefix else name
            safe = sanitize_path(path)
            dt = field.dataType
            
            if isinstance(dt, T.StructType):
                # Nested struct: recursively walk
                walk(dt, path)
            elif isinstance(dt, T.ArrayType) and isinstance(dt.elementType, T.StructType):
                # Array of structs: serialize to JSON to preserve structure
                cols.append(F.to_json(F.col(path)).alias(safe))
            else:
                # Primitive type: direct reference
                cols.append(F.col(path).alias(safe))
    
    walk(df.schema)
    return cols

print("🔧 Column sanitization helper loaded")
print("   Handles: Nested structs, array serialization, special characters")
print()

# ==============================================================================
# SECTION 1: READ ALL JSON FILES FROM VOLUME
# ==============================================================================
# Assumption: JSON files already copied to Unity Catalog volume
# If not, run the download cell first (Step 2 in earlier notebook)
# ==============================================================================

print("="*70)
print("📥 SECTION 1: READING JSON FILES")
print("="*70)
print()
print("📊 Reading all JSON files recursively from volume...")
print(f"   Source: {volume_cves_dir}")
print("   Options: multiLine=True, recursiveFileLookup=True")
print()

t0 = time.time()
df_raw = (
    spark.read
        .option("multiLine", True)          # Handle JSON spanning multiple lines
        .option("recursiveFileLookup", "true")  # Scan subdirectories
        .json(volume_cves_dir)
        .withColumn("_source_file", F.col("_metadata.file_path"))  # Track lineage
)

raw_count = df_raw.count()
load_time = time.time() - t0

print(f"✅ JSON files loaded successfully")
print(f"   - Records: {raw_count:,}")
print(f"   - Load time: {load_time:.1f} seconds")
print(f"   - Avg speed: {raw_count/load_time:,.0f} records/sec")
print()

# ==============================================================================
# SECTION 2: FILTER TO 2024 PUBLICATIONS
# ==============================================================================
# Filtering Logic: Extract year from cveMetadata.datePublished field
# Defensive: Handles null/invalid dates gracefully
# Business Rule: Only analyze 2024 vulnerabilities
# ==============================================================================

print("="*70)
print("📆 SECTION 2: FILTERING TO 2024 RECORDS")
print("="*70)
print()
print("🔍 Filtering by cveMetadata.datePublished year = 2024...")
print("   Technique: to_timestamp() + year() extraction")
print("   Defensive: Handles null dates gracefully")
print()

df_2024 = (
    df_raw
        # Convert datePublished string to timestamp
        .withColumn("datePublished_ts", F.to_timestamp(F.col("cveMetadata.datePublished")))
        # Filter to 2024 year
        .filter(F.year(F.col("datePublished_ts")) == 2024)
        # Add ingestion timestamp for audit trail
        .withColumn("_ingest_ts", F.current_timestamp())
        # Drop temporary timestamp column
        .drop("datePublished_ts")
)

count_2024 = df_2024.count()
filter_pct = (count_2024 / raw_count) * 100 if raw_count > 0 else 0

print(f"✅ 2024 filtering complete")
print(f"   - Raw records: {raw_count:,}")
print(f"   - 2024 records: {count_2024:,}")
print(f"   - Filter rate: {filter_pct:.2f}%")
print()

# ==============================================================================
# SECTION 3: DATA QUALITY CHECKS
# ==============================================================================
# Implements defensive programming and assertion-based validation
# Critical for production pipelines
# ==============================================================================

print("="*70)
print("🧪 SECTION 3: DATA QUALITY VALIDATION")
print("="*70)
print()
print("🔬 Running data quality checks...")
print("   Checks: Null IDs, Duplicates, Record count threshold")
print()

# Count null CVE IDs (should be 0)
null_ids = df_2024.filter(F.col("cveMetadata.cveId").isNull()).count()

# Count distinct CVE IDs (should equal total count)
distinct_ids = df_2024.select("cveMetadata.cveId").distinct().count()

print("📊 Data Quality Summary:")
print(f"   - Total 2024 records:  {count_2024:,}")
print(f"   - Null CVE IDs:        {null_ids:,}")
print(f"   - Distinct CVE IDs:    {distinct_ids:,}")
print(f"   - Duplicate CVE IDs:   {count_2024 - distinct_ids:,}")
print()

print("🧪 Running assertions (will fail pipeline if checks don't pass)...")
try:
    assert null_ids == 0, f"❌ Found {null_ids} null CVE IDs - data quality issue"
    print("   ✅ Null check passed: No null CVE IDs found")
    
    assert distinct_ids == count_2024, f"❌ Found {count_2024 - distinct_ids} duplicate CVE IDs"
    print("   ✅ Uniqueness check passed: All CVE IDs are unique")
    
    assert count_2024 >= 30000, f"❌ Too few 2024 records: {count_2024:,} (expected >= 30,000)"
    print(f"   ✅ Count check passed: {count_2024:,} >= 30,000 records")
    
    print()
    print("✅ All data quality checks passed!")
except AssertionError as e:
    print(f"\n❌ DATA QUALITY FAILURE: {e}")
    raise

print()

# ==============================================================================
# SECTION 4: SCHEMA SANITIZATION
# ==============================================================================
# Problem: Delta Lake doesn't allow special characters in column names
# Solution: Flatten nested structs and sanitize names
# Preserves structure by serializing arrays-of-structs to JSON
# ==============================================================================

print("="*70)
print("🧹 SECTION 4: SCHEMA SANITIZATION")
print("="*70)
print()
print("🔧 Flattening nested schema and sanitizing column names...")
print("   Challenge: JSON has fields like 'Technical Impact' with spaces")
print("   Solution: Replace special chars with underscores")
print("   Preservation: Arrays of structs serialized to JSON strings")
print()

sanitized_cols = build_sanitized_columns(df_2024)
df_clean = df_2024.select(sanitized_cols)

print(f"✅ Schema sanitization complete")
print(f"   - Original columns: {len(df_2024.columns)}")
print(f"   - Sanitized columns: {len(df_clean.columns)}")
print()

# ==============================================================================
# SECTION 5: WRITE TO DELTA LAKE
# ==============================================================================
# Write sanitized data to Delta format
# Benefits: ACID transactions, time travel, schema evolution
# ==============================================================================

print("="*70)
print("💾 SECTION 5: WRITING TO DELTA LAKE")
print("="*70)
print()
print("📊 Writing Bronze Delta table...")
print(f"   Location: {bronze_path}")
print(f"   Records: {count_2024:,}")
print(f"   Format: Delta Lake (ACID compliant)")
print(f"   Mode: Overwrite (idempotent)")
print()

t0 = time.time()
(
    df_clean.write
        .format("delta")
        .mode("overwrite")  # Overwrite for idempotency
        .save(bronze_path)
)
write_time = time.time() - t0

print(f"✅ Delta write complete")
print(f"   - Write time: {write_time:.1f} seconds")
print(f"   - Write speed: {count_2024/write_time:,.0f} records/sec")
print()

# ==============================================================================
# SECTION 6: VERIFICATION
# ==============================================================================
# Read back Delta table to verify successful write
# Show sample data and file structure for assignment screenshots
# ==============================================================================

print("="*70)
print("🔍 SECTION 6: VERIFICATION & SCREENSHOTS")
print("="*70)
print()
print("📊 Reading Delta table back for verification...")
print()

df_check = spark.read.format("delta").load(bronze_path)
check_count = df_check.count()

print(f"✅ Read-back successful")
print(f"   - Records in Delta: {check_count:,}")
print(f"   - Verification: {check_count} == {count_2024} ✓" if check_count == count_2024 else f"   - ⚠️ Count mismatch!")
print()

print("📐 Bronze Delta Schema:")
print("   First 10 columns:")
for i, col in enumerate(df_check.columns[:10], 1):
    print(f"   {i:2d}. {col}")
print(f"   ... and {len(df_check.columns) - 10} more columns")
print()

print("📊 Sample Bronze Records (for screenshot):")
display(df_check.select(
    "cveMetadata_cveId",
    "cveMetadata_datePublished",
    "cveMetadata_state",
    "cveMetadata_assignerShortName",
    "_source_file"
).limit(10))
print()

print("📁 Bronze Delta Files (for screenshot):")
display(dbutils.fs.ls(bronze_path))
print()

print("="*70)
print("📸 REQUIRED SCREENSHOTS FOR ASSIGNMENT")
print("="*70)
print("1. Data Quality Summary showing:")
print("   - Total 2024 records: ~40,000")
print("   - Null CVE IDs: 0")
print("   - All assertions passed")
print()
print("2. df_check.count() output showing ~40,000 records")
print()
print("3. Sample records display showing CVE data")
print()
print("4. dbutils.fs.ls(bronze_path) showing:")
print("   - _delta_log/ directory")
print("   - Multiple .parquet files (partitions)")
print()
print("="*70)
print("✅ BRONZE LAYER COMPLETE - Raw Data Preserved in Delta Lake!")
print("="*70)
print()
print("👉 Next Step: Run 02_bronzetosilver.ipynb for normalization")


🎯 Starting CVE Bronze Layer (FULL RUN – UC Safe)

📖 Reading ALL JSON files recursively from volume...
✅ Raw CVE records loaded: 38,727 files in 2,295.6s

📆 Filtering records for 2024 ...
✅ 2024 CVE records count: 32,924

🧪 Running Data Quality Checks ...
🔍 DQ Summary:
   • Total 2024 rows : 32,924
   • Null CVE IDs   : 0
   • Distinct CVE IDs: 32,924
✅ Basic DQ checks passed.

🧼 Flattening and sanitizing column names ...
✅ Sanitized column count: 104

💾 Writing full 2024 dataset to Delta ...
✅ Delta write complete at: /Volumes/workspace/default/assignment1/bronze

🧾 Reading Delta back for verification ...
✅ Read-back successful. Record count: 32,924

📐 Schema of final Bronze Delta :
root
 |-- containers.adp: string (nullable = true)
 |-- containers.cna.affected: string (nullable = true)
 |-- containers.cna.configurations: string (nullable = true)
 |-- containers.cna.cpeApplicability: string (nullable = true)
 |-- containers.cna.credits: string (nullable = true)
 |-- containers.cna.date

containers.adp,containers.cna.affected,containers.cna.configurations,containers.cna.cpeApplicability,containers.cna.credits,containers.cna.dateAssigned,containers.cna.datePublic,containers.cna.descriptions,containers.cna.exploits,containers.cna.impacts,containers.cna.metrics,containers.cna.problemTypes,containers.cna.providerMetadata.dateUpdated,containers.cna.providerMetadata.orgId,containers.cna.providerMetadata.shortName,containers.cna.references,containers.cna.rejectedReasons,containers.cna.replacedBy,containers.cna.solutions,containers.cna.source.advisory,containers.cna.source.defect,containers.cna.source.defects,containers.cna.source.discovery,containers.cna.source.lang,containers.cna.source.value,containers.cna.tags,containers.cna.timeline,containers.cna.title,containers.cna.workarounds,containers.cna.x_affectedList,containers.cna.x_generator.engine,containers.cna.x_generator.env,containers.cna.x_generator.origin,containers.cna.x_legacyV4Record.CNA_private.Current-Status,containers.cna.x_legacyV4Record.CNA_private.Priority,containers.cna.x_legacyV4Record.CNA_private.STATE,containers.cna.x_legacyV4Record.CNA_private.TYPE,containers.cna.x_legacyV4Record.CNA_private.affectedKeywords,containers.cna.x_legacyV4Record.CNA_private.affectsSummary.affected,containers.cna.x_legacyV4Record.CNA_private.affectsSummary.appliesTo,containers.cna.x_legacyV4Record.CNA_private.affectsSummary.product_versions,containers.cna.x_legacyV4Record.CNA_private.affectsSummary.unaffected,containers.cna.x_legacyV4Record.CNA_private.affectsSummary.unknown,containers.cna.x_legacyV4Record.CNA_private.owner,containers.cna.x_legacyV4Record.CNA_private.publish.month,containers.cna.x_legacyV4Record.CNA_private.publish.year,containers.cna.x_legacyV4Record.CNA_private.publish.ym,containers.cna.x_legacyV4Record.CNA_private.share_with_CVE,containers.cna.x_legacyV4Record.CNA_private.show_cvss,containers.cna.x_legacyV4Record.CVE_data_meta.ASSIGNER,containers.cna.x_legacyV4Record.CVE_data_meta.DATE_PUBLIC,containers.cna.x_legacyV4Record.CVE_data_meta.ID,containers.cna.x_legacyV4Record.CVE_data_meta.STATE,containers.cna.x_legacyV4Record.CVE_data_meta.TITLE,containers.cna.x_legacyV4Record.affects.vendor.vendor_data,containers.cna.x_legacyV4Record.credit,containers.cna.x_legacyV4Record.data_format,containers.cna.x_legacyV4Record.data_type,containers.cna.x_legacyV4Record.data_version,containers.cna.x_legacyV4Record.description.description_data,containers.cna.x_legacyV4Record.exploit,containers.cna.x_legacyV4Record.generator.engine,containers.cna.x_legacyV4Record.impact.cvss.Automatable,containers.cna.x_legacyV4Record.impact.cvss.Recovery,containers.cna.x_legacyV4Record.impact.cvss.Safety,containers.cna.x_legacyV4Record.impact.cvss.attackComplexity,containers.cna.x_legacyV4Record.impact.cvss.attackRequirements,containers.cna.x_legacyV4Record.impact.cvss.attackVector,containers.cna.x_legacyV4Record.impact.cvss.baseScore,containers.cna.x_legacyV4Record.impact.cvss.baseSeverity,containers.cna.x_legacyV4Record.impact.cvss.privilegesRequired,containers.cna.x_legacyV4Record.impact.cvss.providerUrgency,containers.cna.x_legacyV4Record.impact.cvss.subAvailabilityImpact,containers.cna.x_legacyV4Record.impact.cvss.subConfidentialityImpact,containers.cna.x_legacyV4Record.impact.cvss.subIntegrityImpact,containers.cna.x_legacyV4Record.impact.cvss.userInteraction,containers.cna.x_legacyV4Record.impact.cvss.valueDensity,containers.cna.x_legacyV4Record.impact.cvss.vectorString,containers.cna.x_legacyV4Record.impact.cvss.version,containers.cna.x_legacyV4Record.impact.cvss.vulnAvailabilityImpact,containers.cna.x_legacyV4Record.impact.cvss.vulnConfidentialityImpact,containers.cna.x_legacyV4Record.impact.cvss.vulnIntegrityImpact,containers.cna.x_legacyV4Record.impact.cvss.vulnerabilityResponseEffort,containers.cna.x_legacyV4Record.problemtype.problemtype_data,containers.cna.x_legacyV4Record.references.reference_data,containers.cna.x_legacyV4Record.solution,containers.cna.x_legacyV4Record.so


📁 Files in Bronze Delta path :


path,name,size,modificationTime
dbfs:/Volumes/workspace/default/assignment1/bronze/_delta_log/,_delta_log/,0,1762836593612
dbfs:/Volumes/workspace/default/assignment1/bronze/part-00000-e8071914-0002-42b1-999b-f9257168ff28.c000.snappy.parquet,part-00000-e8071914-0002-42b1-999b-f9257168ff28.c000.snappy.parquet,3698405,1762836590000
dbfs:/Volumes/workspace/default/assignment1/bronze/part-00001-d2b889da-10eb-4c33-92e0-9cbace545568.c000.snappy.parquet,part-00001-d2b889da-10eb-4c33-92e0-9cbace545568.c000.snappy.parquet,4957383,1762836590000
dbfs:/Volumes/workspace/default/assignment1/bronze/part-00002-90c6ea31-75df-4216-8adc-01a564b0892d.c000.snappy.parquet,part-00002-90c6ea31-75df-4216-8adc-01a564b0892d.c000.snappy.parquet,4903939,1762836590000
dbfs:/Volumes/workspace/default/assignment1/bronze/part-00003-a50c1550-5552-42b0-9dc5-c4b385a0bb96.c000.snappy.parquet,part-00003-a50c1550-5552-42b0-9dc5-c4b385a0bb96.c000.snappy.parquet,4740399,1762836590000
dbfs:/Volumes/workspace/default/assignment1/bronze/part-00004-5ec1fac5-8431-4267-8f08-7661ba2f9c6f.c000.snappy.parquet,part-00004-5ec1fac5-8431-4267-8f08-7661ba2f9c6f.c000.snappy.parquet,3651867,1762836590000



📸 REQUIRED SCREENSHOTS:
  • df_check.count() ≈ 38 000–40 000 records
  • DQ Summary output above
  • dbutils.fs.ls(bronze_path) showing _delta_log + .parquet files


# Assignment 1: Bronze Layer Example - CVE 2024 Data Ingestion
**DIC 587 - Data Intensive Computing - Fall 2025**

This notebook implements the Bronze layer of our medallion architecture:
- Downloads CVEProject/cvelistV5 repository 
- Filters to 2024 vulnerabilities only (~40,000 records)
- Stores raw JSON as Delta tables with ACID guarantees
- Demonstrates streaming data ingestion patterns

**Learning Objectives:**
- Understand Bronze layer concepts (raw data preservation)
- Practice JSON schema-on-read with PySpark
- Learn Delta Lake table registration
- Handle large-scale data downloads programmatically

## Step 1: Environment Setup and Data Download

In [0]:
import io, os, zipfile, urllib.request, json, shutil
from pyspark.sql.functions import input_file_name, col, to_timestamp, year

# Configuration
USE_OFFLINE = False  # Set True if you manually uploaded /FileStore/cvelistV5.zip
TARGET_YEAR = 2024   # Focus on 2024 CVEs only

# Paths - using /tmp and /FileStore to avoid Community Edition I/O errors
base_dir = "/tmp/cve"
bronze_path = "dbfs:/FileStore/cve/bronze"
os.makedirs(base_dir, exist_ok=True)

print(f"🎯 Target: {TARGET_YEAR} CVE data (~40,000 records)")
print(f"📁 Bronze location: {bronze_path}")

🎯 Target: 2024 CVE data (~40,000 records)
📁 Bronze location: dbfs:/FileStore/cve/bronze


## Step 2: Download CVE Repository

This uploads the entire CVE catalog into a databricks catalog.  You need to figure out how to filter it down to the 2024 records.  You also need to do the other medallion letters!

In [0]:
  import os
  import zipfile
  import urllib.request
  import shutil
  from pyspark.sql.functions import col, to_timestamp, year

  # Configuration
  USE_OFFLINE = False
  TARGET_YEAR = 2024
  volume_root = "/Volumes/workspace/default/assignment1"
  volume_cves_dir = f"{volume_root}/cve/json_files"

  # Check if data already exists in volume FIRST (before downloading!)
  skip_all = False
  try:
      existing_files = dbutils.fs.ls(volume_cves_dir)
      file_count = len([f for f in existing_files if f.name.endswith('.json')])
      
      if file_count > 0:
          print("="*60)
          print(f"✅ Found {file_count} existing JSON files in volume")
          print(f"   📁 Location: {volume_cves_dir}")
          print(f"   ⏩ SKIPPING download, extraction, and copy")
          print(f"   💡 To force re-download, delete the volume directory first:")
          print(f"      dbutils.fs.rm('{volume_cves_dir}', recurse=True)")
          print("="*60)
          print()
          skip_all = True
  except:
      print("📋 No existing data found in volume - proceeding with download...")
      pass

  # Only download and extract if we don't have data
  if not skip_all:
      # Your existing download/extract pattern
      zip_dest = "/tmp/cve/cvelistV5.zip"
      src_extract_dir = "/tmp/cve/cvelistV5-main"

      # Create local temp directory
      os.makedirs("/tmp/cve", exist_ok=True)

      if not USE_OFFLINE:
          zip_url = "https://github.com/CVEProject/cvelistV5/archive/refs/heads/main.zip"
          print(f"🌐 Downloading: {zip_url}")
          print("📈 This is live streaming data - updates every ~7 minutes!")

          with urllib.request.urlopen(zip_url) as r:
              data = r.read()
          with open(zip_dest, "wb") as f:
              f.write(data)

          print(f"✅ Downloaded {len(data):,} bytes")
      else:
          zip_dest = "/dbfs/FileStore/cvelistV5.zip"  # uploaded by you
          assert os.path.exists(zip_dest), "❌ Upload /FileStore/cvelistV5.zip or set USE_OFFLINE=False"
          print("📁 Using offline ZIP file")

      # Clean previous extracts
      if os.path.exists(src_extract_dir):
          shutil.rmtree(src_extract_dir, ignore_errors=True)

      # Extract ZIP
      print("📦 Extracting ZIP archive...")
      with zipfile.ZipFile(zip_dest) as z:
          z.extractall("/tmp/cve/")

      src_glob = "/tmp/cve/cvelistV5-main/cves"
      print(f"📂 Found local JSON directory: {src_glob}")

      src_glob = "/tmp/cve/cvelistV5-main/cves"
      print(f"📂 Found local JSON directory: {src_glob}")

      # Copy JSON files to Unity Catalog volume for Spark access
      print("📋 Copying JSON files to Unity Catalog volume...")
      print("   (Required because Spark cannot access local filesystem)")

      # Remove existing volume directory
      try:
          dbutils.fs.rm(volume_cves_dir, recurse=True)
      except:
          pass

      dbutils.fs.mkdirs(volume_cves_dir)

      # Copy ONLY 2024 JSON files from local to volume (optimized!)
      json_count = 0
      
      # Target only the 2024 directory for efficiency
      target_2024_dir = os.path.join(src_glob, "2024")      
      if os.path.exists(target_2024_dir):
          print(f"   📂 Found 2024 directory: {target_2024_dir}")
          print(f"   🎯 Only copying 2024 CVEs for efficiency...")
          
          for root, dirs, files in os.walk(target_2024_dir):
              for file in files:
                  if file.endswith('.json'):
                      local_file_path = os.path.join(root, file)

                      # Read file content
                      with open(local_file_path, 'r', encoding='utf-8') as f:
                          content = f.read()

                      # Write to volume with unique name (include subdirectory info)
                      relative_path = os.path.relpath(local_file_path, src_glob)
                      safe_filename = relative_path.replace('/', '_').replace('\\', '_')
                      volume_file_path = f"{volume_cves_dir}/{safe_filename}"

                      dbutils.fs.put(volume_file_path, content, overwrite=True)
                      json_count += 1

                      if json_count % 1000 == 0:
                          print(f"      Copied {json_count} JSON files...")
      else:
          print(f"   ⚠️  2024 directory not found, copying all CVEs (slower)...")
          # Fallback: copy all files if 2024 directory doesn't exist
          for root, dirs, files in os.walk(src_glob):
              for file in files:
                  if file.endswith('.json'):
                      local_file_path = os.path.join(root, file)

                      # Read file content
                      with open(local_file_path, 'r', encoding='utf-8') as f:
                          content = f.read()

                      # Write to volume with unique name (include subdirectory info)
                      relative_path = os.path.relpath(local_file_path, src_glob)
                      safe_filename = relative_path.replace('/', '_').replace('\\', '_')
                      volume_file_path = f"{volume_cves_dir}/{safe_filename}"

                      dbutils.fs.put(volume_file_path, content, overwrite=True)
                      json_count += 1

                      if json_count % 1000 == 0:
                          print(f"      Copied {json_count} JSON files...")

      print(f"✅ Copied {json_count} JSON files to volume (2024 CVEs only)")

  # Read the data (whether existing or just copied)
  print("🔄 Reading JSON files with Spark...")

  # Now read from volume location (accessible to Spark)
  df_raw = (spark.read
            .option("multiLine", True)
            .json(volume_cves_dir))

  # Add source file tracking (Unity Catalog compatible)
  df_raw = df_raw.withColumn("_source_file", col("_metadata.file_path"))

  print(f"📊 Total CVE records found: {df_raw.count():,}")


📋 No existing data found in volume - proceeding with download...
🌐 Downloading: https://github.com/CVEProject/cvelistV5/archive/refs/heads/main.zip
📈 This is live streaming data - updates every ~7 minutes!
✅ Downloaded 523,000,347 bytes
📦 Extracting ZIP archive...
📂 Found local JSON directory: /tmp/cve/cvelistV5-main/cves
📂 Found local JSON directory: /tmp/cve/cvelistV5-main/cves
📋 Copying JSON files to Unity Catalog volume...
   (Required because Spark cannot access local filesystem)
   📂 Found 2024 directory: /tmp/cve/cvelistV5-main/cves/2024
   🎯 Only copying 2024 CVEs for efficiency...
Wrote 9943 bytes.
Wrote 5334 bytes.
Wrote 15645 bytes.
Wrote 10115 bytes.
Wrote 5284 bytes.
Wrote 7956 bytes.
Wrote 10339 bytes.
Wrote 5354 bytes.
Wrote 8050 bytes.
Wrote 5162 bytes.
Wrote 4928 bytes.
Wrote 4846 bytes.
Wrote 5436 bytes.
Wrote 6664 bytes.
Wrote 4815 bytes.
Wrote 8366 bytes.
Wrote 5073 bytes.
Wrote 4966 bytes.
Wrote 5139 bytes.
Wrote 10116 bytes.
Wrote 5768 bytes.
Wrote 5398 bytes.
Wro

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
len(dbutils.fs.ls("/Volumes/workspace/default/assignment1/cve/json_files/"))

38727